***In the name of God***


#**Project Report: Medical Transcript Classification System**
**Student: : Mohammad Reza Baghestani**

*   Supervisor: Dr. Amir Zamanidoost
*   date : September 11, 2025

All required libraries are imported here to keep the code clean and organized. This includes data handling with pandas, text processing using string and Counter, machine learning with scikit-learn components and XGBoost, progress tracking through tqdm, and file uploads with google.colab.files.

In [1]:
!pip install pandas scikit-learn numpy tqdm xgboost
import pandas as pd
import string
from collections import Counter
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.preprocessing import normalize
from tqdm import tqdm
from google.colab import files

# Section 1: Upload and Load Data  
This section covers how to upload CSV files (train.csv, valid.csv, test.csv) to Colab and load them into pandas DataFrames. It extracts the text and label columns, changes the labels to 0-based indexing for model compatibility, and checks the data lengths to ensure consistency and avoid runtime errors.

In [2]:
uploaded = files.upload()
for filename in uploaded.keys():
    print(f'File {filename} uploaded successfully, size: {len(uploaded[filename])} bytes')

try:
    train_df = pd.read_csv('train.csv', skip_blank_lines=True)
    val_df = pd.read_csv('valid.csv', skip_blank_lines=True)
    test_df = pd.read_csv('test.csv', skip_blank_lines=True)
    assert all(col in train_df.columns for col in ['label', 'text']), "Missing columns in train.csv"
    assert all(col in val_df.columns for col in ['label', 'text']), "Missing columns in valid.csv"
    assert all(col in test_df.columns for col in ['label', 'text']), "Missing columns in test.csv"
except Exception as e:
    print(f"Error loading data: {e}")
    raise

train_transcripts = train_df['text'].tolist()
val_transcripts = val_df['text'].tolist()
test_transcripts = test_df['text'].tolist()

train_classes = [label - 1 for label in train_df['label'].tolist()]
val_classes = [label - 1 for label in val_df['label'].tolist()]
test_classes = [label - 1 for label in test_df['label'].tolist()]

assert len(train_transcripts) == len(train_classes), "Train data mismatch"
assert len(val_transcripts) == len(val_classes), "Val data mismatch"
assert len(test_transcripts) == len(test_classes), "Test data mismatch"

Saving test.csv to test (2).csv
Saving train.csv to train (2).csv
Saving valid.csv to valid (2).csv
File test (2).csv uploaded successfully, size: 1646779 bytes
File train (2).csv uploaded successfully, size: 12832246 bytes
File valid (2).csv uploaded successfully, size: 1608281 bytes


# Section 2: Preprocess Text  
This section defines a function to preprocess each medical transcript by converting it to lowercase and removing punctuation. This cleaning is necessary before vectorization, as required by the project. The function is applied to all datasets, ensuring consistent input for the next steps.

In [3]:
def clean_transcript(text):
    if not isinstance(text, str):
        return ''
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

train_transcripts_clean = [clean_transcript(text) for text in train_transcripts]
val_transcripts_clean = [clean_transcript(text) for text in val_transcripts]
test_transcripts_clean = [clean_transcript(text) for text in test_transcripts]

# Section 3: Build Vocabulary and Generate Frequency Report  
This section builds the vocabulary from the cleaned training transcripts only. It sorts words by frequency in descending order and selects the top 10,000 most frequent words. It generates a word_freq_report.txt file that includes each word and its frequency count. It also prints a summary of the top 5 words for immediate review.

In [4]:
all_training_words = ' '.join(train_transcripts_clean).split()
word_frequency = Counter(all_training_words)
top_10000_terms = sorted(word_frequency.items(), key=lambda x: x[1], reverse=True)[:10000]
vocabulary_dict = {term: index + 1 for index, (term, _) in enumerate(top_10000_terms)}

with open('word_freq_report.txt', 'w') as f:
    f.write("Term\tFrequency\n")
    for term, freq in top_10000_terms:
        f.write(f"{term}\t{freq}\n")

print("Vocabulary constructed with top 10,000 terms sorted by frequency descending.")
print("Sample of top 5 terms and frequencies:")
for term, freq in top_10000_terms[:5]:
    print(f"{term}: {freq}")
print("Full frequency report saved in 'word_freq_report.txt'.")

Vocabulary constructed with top 10,000 terms sorted by frequency descending.
Sample of top 5 terms and frequencies:
the: 117063
and: 66785
was: 56124
of: 48284
to: 40591
Full frequency report saved in 'word_freq_report.txt'.


# Section 4: Text Vectorization  
This section converts each cleaned medical transcript into a dictionary with exactly 10,000 entries. Each entry shows how often the related vocabulary term appears, using 0 if the term is not present. This results in a 10,000-feature vector for each transcript. This step prepares the data for BBoW and FBoW methods.

In [5]:
def vectorize_to_feature_dict(transcript, vocabulary, feature_size=10000):
    feature_vector = [0] * feature_size
    terms = transcript.split()
    term_count = Counter(terms)
    for term, count in term_count.items():
        if term in vocabulary:
            feature_vector[vocabulary[term] - 1] = count
    return dict(enumerate(feature_vector))

train_feature_vectors = [vectorize_to_feature_dict(transcript, vocabulary_dict) for transcript in train_transcripts_clean]
val_feature_vectors = [vectorize_to_feature_dict(transcript, vocabulary_dict) for transcript in val_transcripts_clean]
test_feature_vectors = [vectorize_to_feature_dict(transcript, vocabulary_dict) for transcript in test_transcripts_clean]

print("Sample feature vector for first train transcript (first 5 features):", dict(list(train_feature_vectors[0].items())[:5]))

Sample feature vector for first train transcript (first 5 features): {0: 1, 1: 2, 2: 0, 3: 1, 4: 1}


# Section 5: Implement BBoW and FBoW Methods  
This section uses the Binary Bag-of-Words (BBoW) method. It converts frequency vectors to binary presence where 1 indicates the term appears and 0 shows it does not. The Frequency Bag-of-Words (FBoW) method normalizes frequencies so they sum to 1. Vectors are changed into matrices for input into the model.

In [6]:
def convert_vectors_to_matrix(feature_vectors, feature_size=10000):
    return np.array([list(v.values()) for v in feature_vectors])

X_train_bbow_matrix = (convert_vectors_to_matrix(train_feature_vectors) > 0).astype(int)
X_val_bbow_matrix = (convert_vectors_to_matrix(val_feature_vectors) > 0).astype(int)
X_test_bbow_matrix = (convert_vectors_to_matrix(test_feature_vectors) > 0).astype(int)

X_train_fbow_matrix = convert_vectors_to_matrix(train_feature_vectors)
X_val_fbow_matrix = convert_vectors_to_matrix(val_feature_vectors)
X_test_fbow_matrix = convert_vectors_to_matrix(test_feature_vectors)
X_train_fbow_matrix = normalize(X_train_fbow_matrix, norm='l1', axis=1)
X_val_fbow_matrix = normalize(X_val_fbow_matrix, norm='l1', axis=1)
X_test_fbow_matrix = normalize(X_test_fbow_matrix, norm='l1', axis=1)

train_classes_array = np.array(train_classes)
val_classes_array = np.array(val_classes)
test_classes_array = np.array(test_classes)

# Section 6: Train and Evaluate Models  
This section trains Logistic Regression, Decision Tree, Random Forest, and XGBoost models. It uses BBoW and FBoW representations and tunes hyperparameters with GridSearchCV. Performance is evaluated using the macro F1-score on the train, validation, and test sets. Detailed results are provided.

In [ ]:
def train_and_evaluate(model, hyperparams_grid, X_train, y_train, X_val, y_val, X_test, y_test, model_name):
    print(f"Starting training for {model_name}...")
    grid_search = GridSearchCV(model, hyperparams_grid, cv=3, scoring='f1_macro', n_jobs=1)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_

    y_train_pred = best_model.predict(X_train)
    y_val_pred = best_model.predict(X_val)
    y_test_pred = best_model.predict(X_test)

    f1_train = f1_score(y_train, y_train_pred, average='macro')
    f1_val = f1_score(y_val, y_val_pred, average='macro')
    f1_test = f1_score(y_test, y_test_pred, average='macro')

    print(f"Completed {model_name}")
    return f1_train, f1_val, f1_test, grid_search.best_params_

model_configs = {
    'LogisticRegression': (LogisticRegression(max_iter=2000, solver='liblinear'), {'C': [0.01, 0.1, 1, 10, 100]}),
    'DecisionTree': (DecisionTreeClassifier(), {'max_depth': [3, 5, 10, 20, None], 'min_samples_split': [2, 5, 10]}),
    'RandomForest': (RandomForestClassifier(n_jobs=1), {'n_estimators': [50, 100, 200], 'max_depth': [5, 10, 20, None]}),
    'XGBoost': (XGBClassifier(eval_metric='mlogloss', n_jobs=1), {'learning_rate': [0.01, 0.05, 0.1], 'n_estimators': [50, 100, 200], 'max_depth': [3, 5, 7]})
}

bbow_results = {}
for name, (model, param_grid) in model_configs.items():
    bbow_results[name] = train_and_evaluate(model, param_grid, X_train_bbow_matrix, train_classes_array, X_val_bbow_matrix, val_classes_array, X_test_bbow_matrix, test_classes_array, name)

fbow_results = {}
for name, (model, param_grid) in model_configs.items():
    fbow_results[name] = train_and_evaluate(model, param_grid, X_train_fbow_matrix, train_classes_array, X_val_fbow_matrix, val_classes_array, X_test_fbow_matrix, test_classes_array, name)

Starting training for LogisticRegression...
Completed LogisticRegression
Starting training for DecisionTree...
Completed DecisionTree
Starting training for RandomForest...
Completed RandomForest
Starting training for XGBoost...
Completed XGBoost
Starting training for LogisticRegression...
Completed LogisticRegression
Starting training for DecisionTree...
Completed DecisionTree
Starting training for RandomForest...
Completed RandomForest
Starting training for XGBoost...


# Section 7: Report F1-Scores  
This section shows the F1-scores for the training, validation, and test sets for both BBoW and FBoW representations. It presents this information in a structured format and includes the best hyperparameters for each model.

In [ ]:
print("BBoW Results:")
for name, (f1_train, f1_val, f1_test, params) in bbow_results.items():
    print(f"{name}: Train F1={f1_train:.4f}, Val F1={f1_val:.4f}, Test F1={f1_test:.4f}, Best Params={params}")

print("\nFBoW Results:")
for name, (f1_train, f1_val, f1_test, params) in fbow_results.items():
    print(f"{name}: Train F1={f1_train:.4f}, Val F1={f1_val:.4f}, Test F1={f1_test:.4f}, Best Params={params}")

# Section 8: Generate Deliverable Files  
This section creates vocab.txt in the format (word\tID\tfreq) and data files (train.txt, val.txt, test.txt) with space-separated word IDs and 1-based labels at the end. It ignores unknown words according to project instructions.

In [ ]:
with open('vocab.txt', 'w') as f:
    for word, idx in sorted(vocabulary_dict.items(), key=lambda x: x[1]):
        freq = word_frequency.get(word, 0)
        f.write(f"{word}\t{idx}\t{freq}\n")

def generate_data_file(filename, cleaned_transcripts, class_labels):
    with open(filename, 'w') as f:
        for transcript, label in tqdm(zip(cleaned_transcripts, class_labels), total=len(cleaned_transcripts), desc=f"Generating {filename}"):
            tokens = transcript.split() if transcript else []
            ids = [str(vocabulary_dict.get(token, '')) for token in tokens if token in vocabulary_dict]
            line = ' '.join(ids) + f" {label + 1}\n"
            f.write(line)

generate_data_file('train.txt', train_transcripts_clean, train_classes)
generate_data_file('val.txt', val_transcripts_clean, val_classes)
generate_data_file('test.txt', test_transcripts_clean, test_classes)